# Human numbers

In [2]:
from fastai.text import *

In [3]:
bs=64

## Data

Untar the data at `URLs.HUMAN_NUMBERS` and save the path to `path`. Show the files in `path`.

In [4]:
path = untar_data(URLs.HUMAN_NUMBERS)
path.ls()

[PosixPath('/home/paperspace/.fastai/data/human_numbers/train.txt'),
 PosixPath('/home/paperspace/.fastai/data/human_numbers/valid.txt')]

Define a function `readnums` that takes in an argument `d` and returns the lines in `path/d` separated by ', '.

In [5]:
def readnums(d): return [', '.join(o.strip() for o in open(path/d).readlines())]

Use `readnums` to read `train.txt` and show the first line and the first 80 values, saving it to `train_txt`.

In [6]:
train_txt = readnums('train.txt'); train_txt[0][:80]

'one, two, three, four, five, six, seven, eight, nine, ten, eleven, twelve, thirt'

Do the same for the validation, only this time we'll look for the _last_ 80 values in the first row. Save it to `valid_txt`.

In [7]:
valid_txt = readnums('valid.txt'); valid_txt[0][-80:]

' nine thousand nine hundred ninety eight, nine thousand nine hundred ninety nine'

Create a `TextList` from each of `train_txt` and `val_txt`, using `path`. Create `src`, an `ItemLists`, from `path`, `train`, and `valid` (the two `TextList`s that you just created). Label it for a language model (see `label_for_lm`). Create a databunch where batch size is equal to `bs` (defined above).

In [8]:
train = TextList(train_txt, path=path)
valid = TextList(valid_txt, path=path)

src = ItemLists(path=path, train=train, valid=valid).label_for_lm()
data = src.databunch(bs=bs)

Index into the training data and visualize it using the `.text` property. Show the first 80 characters of the first example in the training set.

In [9]:
train[0].text[:80]

'xxbos one , two , three , four , five , six , seven , eight , nine , ten , eleve'

Show the length of the `x` portion of the first example in the validation set. Hint: check out the property `data`. On the example. What does this do?

In [10]:
len(data.valid_ds[0][0].data)

13017

Show a tuple of the `data.bptt` property, and the length of `data.valid_dl`. Why is the length of the valid_dl what it is? 

In [11]:
data.bptt, len(data.valid_dl)

(70, 3)

Divide the length of the first validation example by the bptt size and it should equal a little less than the length of the validation data loader.

In [12]:
13017/70/bs

2.905580357142857

Get an iterator of the `valid_dl` and call it `it`. Call `next` and set it to `x1,y1`. Do it again and call it `x2,y2`. Again and call it `x3,y3`. Then close the iterator.

In [13]:
it = iter(data.valid_dl)
x1,y1 = next(it)
x2,y2 = next(it)
x3,y3 = next(it)
it.close()

Show the textified version of `x1[0]`, and then the textified version of `y1[0]`. 

In [31]:
data.vocab.textify(x1[:,0])

'xxbos , thousand one two thousand hundred , hundred three eight sixty eight four five thousand eight , hundred , eight sixty eight thousand eight thousand hundred , eight nine thousand seventy nine forty nine nine fifty nine thousand three thousand thousand , three , nine hundred nine sixteen six thousand hundred , six , nine hundred nine thousand nine thousand nine nine nine'

In [32]:
data.vocab.textify(y1[:,0])

'eight eight eighty hundred , one thirteen eight seventy hundred thousand four thousand hundred , four thousand eight seventy eight thousand seven thousand seven , seven twenty eight hundred hundred nine , thousand five thousand thousand two thousand two , two three nine hundred nine thousand fifty thousand , , five six nine hundred nine thousand fifty thousand eight , eight thousand thousand hundred'

Do the same for `x2[0]` and `y2[0]`.

In [16]:
data.vocab.textify([i for i in x2[0]])

'thousand eighteen , eight thousand nineteen , eight thousand twenty , eight thousand twenty one , eight thousand twenty two , eight thousand twenty three , eight thousand twenty four , eight thousand twenty five , eight thousand twenty six , eight thousand twenty seven , eight thousand twenty eight , eight thousand twenty nine , eight thousand thirty , eight thousand thirty one , eight thousand thirty two ,'

In [17]:
data.vocab.textify([i for i in y2[0]])

'eighteen , eight thousand nineteen , eight thousand twenty , eight thousand twenty one , eight thousand twenty two , eight thousand twenty three , eight thousand twenty four , eight thousand twenty five , eight thousand twenty six , eight thousand twenty seven , eight thousand twenty eight , eight thousand twenty nine , eight thousand thirty , eight thousand thirty one , eight thousand thirty two , eight'

Now do `x1[1]` and `y1[1]`.

In [18]:
data.vocab.textify([i for i in x1[1]])

', eight thousand forty six , eight thousand forty seven , eight thousand forty eight , eight thousand forty nine , eight thousand fifty , eight thousand fifty one , eight thousand fifty two , eight thousand fifty three , eight thousand fifty four , eight thousand fifty five , eight thousand fifty six , eight thousand fifty seven , eight thousand fifty eight , eight thousand fifty nine ,'

In [19]:
data.vocab.textify([i for i in y1[1]])

'eight thousand forty six , eight thousand forty seven , eight thousand forty eight , eight thousand forty nine , eight thousand fifty , eight thousand fifty one , eight thousand fifty two , eight thousand fifty three , eight thousand fifty four , eight thousand fifty five , eight thousand fifty six , eight thousand fifty seven , eight thousand fifty eight , eight thousand fifty nine , eight'

Show the shape of x1.

In [20]:
x1.shape

torch.Size([64, 70])

What have you noticed about the structure of the data?

In [75]:
# Each batch is of length ${batch_size}, while each example is of length. ${bptt}.
# In this case, x1 is length 64 where each element in x1 is a length-70 vector of tokens.
# As such, each batch is a 64x70 matrix.
# x2[0] is the continuation of x1[0], while x1[1] is the next length-70 block of text after xn[0].
# Basically, the first example from all of the batches glues together forms one logical string of text,
# which context to the second example from all of the batches, and so on. The logical ordering of text
# is between the same index across different batches. In other words, if there are 100 batches, then the first
# 100 strings of text are x1[0],x2[0],...,x100[0], and then we move onto x1[1],x2[1],...,x100[1], etc.


In [22]:
x1[0]

tensor([ 2, 19, 11, 12,  9, 19, 11, 13,  9, 19, 11, 14,  9, 19, 11, 15,  9, 19,
        11, 16,  9, 19, 11, 17,  9, 19, 11, 18,  9, 19, 11, 19,  9, 19, 11, 20,
         9, 19, 11, 29,  9, 19, 11, 30,  9, 19, 11, 31,  9, 19, 11, 32,  9, 19,
        11, 33,  9, 19, 11, 34,  9, 19, 11, 35,  9, 19, 11, 36,  9, 19],
       device='cuda:0')

Show the total number of elements in the whole dataset.

In [23]:
x1.numel()+x2.numel()+x3.numel()

13440

Show the shapes of x1 and y1.

In [24]:
x1.shape,y1.shape

(torch.Size([64, 70]), torch.Size([64, 70]))

Show the shapes of x2 and y2.

In [25]:
x2.shape,y2.shape

(torch.Size([64, 70]), torch.Size([64, 70]))

Show `x1[0]` and `y1[0]`. What do you notice about them?

In [34]:
x1[0]

tensor([ 2, 19, 11, 12,  9, 19, 11, 13,  9, 19, 11, 14,  9, 19, 11, 15,  9, 19,
        11, 16,  9, 19, 11, 17,  9, 19, 11, 18,  9, 19, 11, 19,  9, 19, 11, 20,
         9, 19, 11, 29,  9, 19, 11, 30,  9, 19, 11, 31,  9, 19, 11, 32,  9, 19,
        11, 33,  9, 19, 11, 34,  9, 19, 11, 35,  9, 19, 11, 36,  9, 19],
       device='cuda:0')

In [35]:
y1[0]

tensor([19, 11, 12,  9, 19, 11, 13,  9, 19, 11, 14,  9, 19, 11, 15,  9, 19, 11,
        16,  9, 19, 11, 17,  9, 19, 11, 18,  9, 19, 11, 19,  9, 19, 11, 20,  9,
        19, 11, 29,  9, 19, 11, 30,  9, 19, 11, 31,  9, 19, 11, 32,  9, 19, 11,
        33,  9, 19, 11, 34,  9, 19, 11, 35,  9, 19, 11, 36,  9, 19, 11],
       device='cuda:0')

In [39]:
# The `y` value is the `x` example left-shifted by one.

Before each of the following, try to take a guess at which data is going to be captured. It might be helpful to have a way to draw it out on the side.

Textify the x1[0] and y1[0].

In [40]:
data.train_ds.vocab.textify(x1[0])

'xxbos eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen , eight thousand seventeen , eight'

In [41]:
data.train_ds.vocab.textify(y1[0])

'eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen , eight thousand seventeen , eight thousand'

Set `v` to `data.valid_ds.vocab`.

In [38]:
v = data.valid_ds.vocab

Textify `x1[0]`.

In [ ]:
v.textify(x1[0])

'xxbos eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen , eight thousand seventeen , eight'

Textify `y1[0]`.

In [ ]:
v.textify(y1[0])

'eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen , eight thousand seventeen , eight thousand'

Textify `x2[0]`.

In [ ]:
v.textify(x2[0])

'thousand eighteen , eight thousand nineteen , eight thousand twenty , eight thousand twenty one , eight thousand twenty two , eight thousand twenty three , eight thousand twenty four , eight thousand twenty five , eight thousand twenty six , eight thousand twenty seven , eight thousand twenty eight , eight thousand twenty nine , eight thousand thirty , eight thousand thirty one , eight thousand thirty two ,'

Texxtify `x3[0]`.

In [ ]:
v.textify(x3[0])

'eight thousand thirty three , eight thousand thirty four , eight thousand thirty five , eight thousand thirty six , eight thousand thirty seven , eight thousand thirty eight , eight thousand thirty nine , eight thousand forty , eight thousand forty one , eight thousand forty two , eight thousand forty three , eight thousand forty four , eight thousand forty five , eight thousand forty six , eight'

Textify `x1[1]`.

In [ ]:
v.textify(x1[1])

', eight thousand forty six , eight thousand forty seven , eight thousand forty eight , eight thousand forty nine , eight thousand fifty , eight thousand fifty one , eight thousand fifty two , eight thousand fifty three , eight thousand fifty four , eight thousand fifty five , eight thousand fifty six , eight thousand fifty seven , eight thousand fifty eight , eight thousand fifty nine ,'

Textify `x2[1]`.

In [ ]:
v.textify(x2[1])

'eight thousand sixty , eight thousand sixty one , eight thousand sixty two , eight thousand sixty three , eight thousand sixty four , eight thousand sixty five , eight thousand sixty six , eight thousand sixty seven , eight thousand sixty eight , eight thousand sixty nine , eight thousand seventy , eight thousand seventy one , eight thousand seventy two , eight thousand seventy three , eight thousand'

Textify `x3[1]`.

In [ ]:
v.textify(x3[1])

'seventy four , eight thousand seventy five , eight thousand seventy six , eight thousand seventy seven , eight thousand seventy eight , eight thousand seventy nine , eight thousand eighty , eight thousand eighty one , eight thousand eighty two , eight thousand eighty three , eight thousand eighty four , eight thousand eighty five , eight thousand eighty six , eight thousand eighty seven , eight thousand eighty'

Textify the last example in `x3`.

In [ ]:
v.textify(x3[-1])

'ninety , nine thousand nine hundred ninety one , nine thousand nine hundred ninety two , nine thousand nine hundred ninety three , nine thousand nine hundred ninety four , nine thousand nine hundred ninety five , nine thousand nine hundred ninety six , nine thousand nine hundred ninety seven , nine thousand nine hundred ninety eight , nine thousand nine hundred ninety nine xxbos eight thousand one , eight'

In [73]:
data.vocab.textify(x1[0])

'xxbos eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen , eight thousand seventeen , eight'

Show a batch from the validation set.

In [ ]:
data.show_batch(ds_type=DatasetType.Valid)

idx,text
0,"thousand forty seven , eight thousand forty eight , eight thousand forty nine , eight thousand fifty , eight thousand fifty one , eight thousand fifty two , eight thousand fifty three , eight thousand fifty four , eight thousand fifty five , eight thousand fifty six , eight thousand fifty seven , eight thousand fifty eight , eight thousand fifty nine , eight thousand sixty , eight thousand sixty"
1,"eight , eight thousand eighty nine , eight thousand ninety , eight thousand ninety one , eight thousand ninety two , eight thousand ninety three , eight thousand ninety four , eight thousand ninety five , eight thousand ninety six , eight thousand ninety seven , eight thousand ninety eight , eight thousand ninety nine , eight thousand one hundred , eight thousand one hundred one , eight thousand one"
2,"thousand one hundred twenty four , eight thousand one hundred twenty five , eight thousand one hundred twenty six , eight thousand one hundred twenty seven , eight thousand one hundred twenty eight , eight thousand one hundred twenty nine , eight thousand one hundred thirty , eight thousand one hundred thirty one , eight thousand one hundred thirty two , eight thousand one hundred thirty three , eight thousand"
3,"three , eight thousand one hundred fifty four , eight thousand one hundred fifty five , eight thousand one hundred fifty six , eight thousand one hundred fifty seven , eight thousand one hundred fifty eight , eight thousand one hundred fifty nine , eight thousand one hundred sixty , eight thousand one hundred sixty one , eight thousand one hundred sixty two , eight thousand one hundred sixty three"
4,"thousand one hundred eighty three , eight thousand one hundred eighty four , eight thousand one hundred eighty five , eight thousand one hundred eighty six , eight thousand one hundred eighty seven , eight thousand one hundred eighty eight , eight thousand one hundred eighty nine , eight thousand one hundred ninety , eight thousand one hundred ninety one , eight thousand one hundred ninety two , eight thousand"


## Single fully connected model

Set `data` to a databunch from `src` with `bs` from above, abd `bptt=3`.

In [58]:
data = src.databunch(bs=bs, bptt=3)

Set `x,y` to the result of a batch and check their shapes.

In [59]:
x,y = data.one_batch()
x.shape,y.shape

(torch.Size([64, 3]), torch.Size([64, 3]))

In [60]:
x.shape

torch.Size([64, 3])

In [61]:
x.shape

torch.Size([64, 3])

In [71]:
data.vocab.textify(x[1])

'two hundred eleven'

Set `nv` to the length of `v.itos`. What is `v.itos`? Print the value.

In [42]:
# itos is the integer-to-string mapping for our vocabulary.
nv = len(v.itos); nv

39

Set `nh` to 64. Can you guess what `nh` is for?

In [43]:
# nh is the size of the hidden layer. 
nh=64

Define a function `loss4` that takes an input and a target and calls `F.cross_entropy` on input and the last column of the target. Similarly do do the same thing for `accuracy` which calls `accuracy` on the last column. Why do we care about the last column?

In [52]:
data.train_ds.vocab.textify(x1[0])

'xxbos eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen , eight thousand seventeen , eight'

In [76]:
# We care about the last column because the targets are all offset by 1, meaning target[:, -1]
# is the last unknown value in each example. After we've built up our hidden layers for `bptt` iterations,
# we have a lot of information built up about the textual context, and so we want to see if we can make a good
# prediction on token bptt + 1.
def loss4(input,target): return F.cross_entropy(input, target[:,-1])
def acc4 (input,target): return accuracy(input, target[:,-1])

Create a class `Model0` the inherits from `nn.Module`. Make sure you call `super.__init__()`. Then, set four layers at the instance level:
- self.i_h should be an embedding layer (nv, nh) 
- self.h_h should be a linear layer (nh, nh) 
- self.h_o should be a linear layer (nh, nv) 
- self.bn should be a batchnorm layer (nh)

`h` should start by calling i_h on the x inputs, along with a relu and a batchnorm. Then, if x.shape[1] > 1, we'll reset `h` as the sum of itself and i_h of x[:,1]. We'll then set it again as the batch-normed relu'd h_h of itself. If x.shape [1] > 2, we'll do this again with x[:,2]. Finally, we'll return h_o of the hidden layer. Can you explain how this works (even if at a high level)?

In [80]:
# The i_h layer is an input-to-hidden layer. It's going to take an index and grab the `nh`-length embedding
# vector that represents that word. This will be applied pointwise to the first column-slice of the batch matrix.
# Then we call relu and batch norm that to get our initial `h` value. Next, we add `h` and the embedding for
# x[:, 1], call h_h on it which keeps the shape the same, and call bn/relu on that value. 
# (Not: relu comes before bn) We then do the same thing for x[:, 2].
# Basically, at each iteration, we add the existing hidden layer to the embedding for the new word,
# call h_h on it to transform it linearly and then call bn/relu on it. The dimensions are as follows: 
# Input: bs x 1
# Hidden: bs x nh (embedding layer transforms each idx into a bs x nv oh-encoded vector and matrix-multiplies
#     by the nv x nh embedding matrix)
#     Then hidden layer is multiplied by nh x nh linear layer resulting in another bs x nh matrix, and relu/bn
#     is performed pointwise
# Output: bs x nv (bs x nh hidden layer is multiplied by nh x nv output matrix and then relu/bn is applied
#     pointwise)
# We do this over x[:,0], x[:,1], and x[:,2], and our dataset only has three columns (bc of the bptt we set above).
# Note that the initial value of the hidden layer is assumed to be a zero-vector.
class Model0(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)  # green arrow
        self.h_h = nn.Linear(nh,nh)     # brown arrow
        self.h_o = nn.Linear(nh,nv)     # blue arrow
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        h = self.bn(F.relu(self.i_h(x[:,0])))
        if x.shape[1]>1:
            h = h + self.i_h(x[:,1])
            h = self.bn(F.relu(self.h_h(h)))
        if x.shape[1]>2:
            h = h + self.i_h(x[:,2])
            h = self.bn(F.relu(self.h_h(h)))
        return self.h_o(h)

Set `learn` to a `Learner` using `data` and a `Model0`, with loss func `loss4` and metrics `acc4`.

In [78]:
learn = Learner(data, Model0(), loss_func=loss4, metrics=acc4)

Fit a cycle with 6 epochs at a learning rate of `1e-4`.

In [79]:
learn.fit_one_cycle(6, 1e-4)

epoch,train_loss,valid_loss,acc4,time
0,3.646418,3.553160,0.095818,00:01
1,3.094123,3.170548,0.366728,00:01
2,2.472620,2.719015,0.450138,00:01
3,2.138093,2.454260,0.465533,00:01
4,2.010855,2.352926,0.467142,00:01
5,1.984788,2.337928,0.467601,00:01


## Same thing with a loop

Now write the same thing as above with a loop. Call the class `Model1`. Hint: to initialize a torch zero vector on the GPU, you'll have to call `.to(device=x.device)`.

In [81]:
class Model1(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)  # green arrow
        self.h_h = nn.Linear(nh,nh)     # brown arrow
        self.h_o = nn.Linear(nh,nv)     # blue arrow
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        h = torch.zeros(x.shape[0], nh).to(device=x.device)
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = self.bn(F.relu(self.h_h(h)))
        return self.h_o(h)

In [82]:
learn = Learner(data, Model1(), loss_func=loss4, metrics=acc4)

In [83]:
learn.fit_one_cycle(6, 1e-4)

epoch,train_loss,valid_loss,acc4,time
0,3.629292,3.595804,0.044807,00:01
1,3.098353,3.073421,0.406480,00:01
2,2.519140,2.573483,0.463235,00:01
3,2.174855,2.293261,0.468290,00:01
4,2.040694,2.196173,0.468980,00:01
5,2.012976,2.182310,0.469439,00:01


## Multi fully connected model

Create a new `data` that has `bptt=20`.

In [ ]:
data = src.databunch(bs=bs, bptt=20)

Grab a batch and set it to `x,y`. Show the shapes.

In [ ]:
x,y = data.one_batch()
x.shape,y.shape

(torch.Size([64, 20]), torch.Size([64, 20]))

Write a new module `Model2` that accumulates all of the calls to `h_o` in a list called `res`, and returns a torch-stacked (across the horizontal axis) collection of all of the values in `res`. What would you expect this output to look like? What would you expect the benefit of doing this to be?

In [105]:
# Based on some basic experimentation, I'd expect this to be a rank-3 tensor with dimensions 
# bptt * nv * bs. But I don't fully understand this so we should come back to it.
# Basically, the rows would represent each time slice. The columns represent the probabilities of the
# different tokens in our vocab. The "depth slices" represent the different examples in our batch.
# So element [10, 20, 37] would represent the probability of the 20th token being the next one 
# at the 10th timestep for the 37th example in the batch. 
# I'm *guessing* that the loss function that we use has to have the same shape except the column
# slices have the ground truth instead of probabilities. Everything else about it should be the same.
class Model2(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.h_h = nn.Linear(nh,nh)
        self.h_o = nn.Linear(nh,nv)
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        h = torch.zeros(x.shape[0], nh).to(device=x.device)
        res = []
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
            res.append(self.h_o(self.bn(h)))
        return torch.stack(res, dim=1)

In [106]:
g = torch.tensor([[1,2,3], [4,5,6]])

In [107]:
h = torch.tensor([[7,8,9], [10,11,12]])

In [108]:
l = [g, h]

In [109]:
g

tensor([[1, 2, 3],
        [4, 5, 6]])

In [110]:
h

tensor([[ 7,  8,  9],
        [10, 11, 12]])

In [104]:
torch.stack(l, dim=1)

tensor([[[ 1,  2,  3],
         [ 7,  8,  9]],

        [[ 4,  5,  6],
         [10, 11, 12]]])

Create a learner with `data` and `Model2()` with `metrics=accuracy`.

In [111]:
learn = Learner(data, Model2(), metrics=accuracy)

Fit a cycle at 1e-4 and `pct_start=0.1`.

In [112]:
learn.fit_one_cycle(10, 1e-4, pct_start=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,3.422101,3.437762,0.210248,00:01
1,2.697701,2.922462,0.309360,00:01
2,2.186082,2.544540,0.364966,00:01
3,1.921796,2.384633,0.294041,00:01
4,1.788954,2.321904,0.276731,00:01
5,1.722186,2.304682,0.274663,00:01
6,1.688658,2.304769,0.274280,00:01
7,1.672707,2.307635,0.274586,00:01
8,1.666427,2.309280,0.274510,00:01
9,1.665084,2.309555,0.274510,00:01


The loss is worse; why do you think this is?

In [113]:
# The loss is worse because now a lot of more challenging examples are counted towards our loss; for example,
# we have to predict the second word from the first, and we hardly have any context at all! Every batch we're
# clearing out the hidden layer and losing any information we had about the context up to that point. But remember:
# the batches are linked together; batch0[0] and batch1[0] are sequential pieces of text! So we can hang onto
# the information represented in the hidden layer and use it to propagate that context forward across batches.

## Maintain state

In [ ]:
class Model3(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.h_h = nn.Linear(nh,nh)
        self.h_o = nn.Linear(nh,nv)
        self.bn = nn.BatchNorm1d(nh)
        self.h = torch.zeros(bs, nh).cuda()
        
    def forward(self, x):
        res = []
        h = self.h
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
            res.append(self.bn(h))
        self.h = h.detach()
        res = torch.stack(res, dim=1)
        res = self.h_o(res)
        return res

In [ ]:
learn = Learner(data, Model3(), metrics=accuracy)

In [ ]:
learn.fit_one_cycle(20, 3e-3)

epoch,train_loss,valid_loss,accuracy
1,3.598183,3.556362,0.050710
2,3.274616,2.975699,0.401634
3,2.624206,2.036894,0.467330
4,2.022702,1.956439,0.316193
5,1.681813,1.934952,0.336861
6,1.453007,1.948201,0.351349
7,1.276971,2.005776,0.368679
8,1.138499,2.081261,0.360156
9,1.029217,2.145853,0.360795
10,0.939949,2.215388,0.372230


## nn.RNN

In [ ]:
class Model4(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.rnn = nn.RNN(nh,nh, batch_first=True)
        self.h_o = nn.Linear(nh,nv)
        self.bn = BatchNorm1dFlat(nh)
        self.h = torch.zeros(1, bs, nh).cuda()
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(self.bn(res))

In [ ]:
learn = Learner(data, Model4(), metrics=accuracy)

In [ ]:
learn.fit_one_cycle(20, 3e-3)

epoch,train_loss,valid_loss,accuracy
1,3.451432,3.268344,0.224148
2,2.974938,2.456569,0.466051
3,2.316732,1.946969,0.465625
4,1.866151,1.991952,0.314702
5,1.618516,1.802403,0.437216
6,1.411517,1.731107,0.436293
7,1.171916,1.655979,0.504048
8,0.965887,1.579963,0.522088
9,0.797046,1.479819,0.565057
10,0.659378,1.487831,0.579048


## 2-layer GRU

In [ ]:
class Model5(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.rnn = nn.GRU(nh, nh, 2, batch_first=True)
        self.h_o = nn.Linear(nh,nv)
        self.bn = BatchNorm1dFlat(nh)
        self.h = torch.zeros(2, bs, nh).cuda()
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(self.bn(res))

In [ ]:
learn = Learner(data, Model5(), metrics=accuracy)

In [ ]:
learn.fit_one_cycle(10, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,2.864854,2.314943,0.454545
2,1.798988,1.357116,0.629688
3,0.932729,1.307463,0.796733
4,0.451969,1.329699,0.788636
5,0.225787,1.293570,0.800142
6,0.118085,1.265926,0.803338
7,0.065306,1.207096,0.806960
8,0.038098,1.205361,0.813920
9,0.024069,1.239411,0.807813
10,0.017078,1.253409,0.807102


## fin